In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re

In [11]:
# llegamos a las actividades en Asturias
driver = webdriver.Chrome()             
url = 'https://www.civitatis.com/es/'
driver.get(url)
driver.maximize_window()  
driver.find_element('css selector','#didomi-notice-agree-button > span').click()
driver.find_element('css selector','#home-search-text').click()
sleep(5)
driver.find_element('css selector','#home-search-text').send_keys('asturias',Keys.TAB)
driver.find_element('css selector','#page-header__search-button').click()

In [12]:
pagina = 1
ciudad = 'asturias'


In [18]:
url = 'https://www.civitatis.com/es/asturias/'          #url no dinamica de momento porque la pagina peta
                                                        #literalmente no hace la resq

In [19]:

def carga_datos():
    """_summary_:Cargar datos de actividades desde una página web.

    Realiza una solicitud HTTP a la URL especificada, analiza el contenido HTML
    para extraer información sobre actividades, incluyendo títulos, descripciones
    y precios, y devuelve esta información en un DataFrame de pandas.

    Returns:
        pd.DataFrame: Un DataFrame que contiene tres columnas: 'Actividad', 
        'Descripción' y 'Precio'. Si la solicitud falla o no se encuentran 
        productos, se devuelve un DataFrame vacío.

    """

    res = requests.get(url)
    if res.status_code == 200:
        print("Solicitud exitosa")
    else:
        print("Error en la solicitud")
        return pd.DataFrame()  

    sopa_actividades = BeautifulSoup(res.content, "html.parser")
    lista_productos = sopa_actividades.findAll("div", {'class': 'o-search-list__item'})
    # Listas para almacenar los datos
    titulos = []
    descripciones = []
    precios = []
    for producto in lista_productos:
        # Buscar los elementos
        titulo_elemento = producto.find('h2')
        descripcion_elemento = producto.find("div", {'class': 'comfort-card__text l-list-card__text'})
        precio_elemento = producto.find("span", {'class': 'comfort-card__price__text'})
        
        if titulo_elemento and descripcion_elemento and precio_elemento:
            titulo = titulo_elemento.get_text(strip=True)
            descripcion = descripcion_elemento.get_text(strip=True)
            precio = precio_elemento.get_text(strip=True)
            
            # Añadir los datos a las listas
            titulos.append(titulo)
            descripciones.append(descripcion)
            precios.append(precio)
      
    # Crear un DataFrame con los datos
    df = pd.DataFrame({
        'Actividad': titulos,
        'Descripción': descripciones,
        'Precio': precios,
    })

    return df

# Llamar a la función y mostrar el DataFrame
df_actividades = carga_datos()

df_actividades

Solicitud exitosa


,Actividad,Descripción,Precio
0,"Excursión a Covadonga, Cangas de Onís y la cos...","Si os apasionala historia y la naturaleza, no ...",35€
1,"Excursión a Avilés, Cudillero y Luarca",Acompañadnos a descubrir el hermoso ycautivado...,35€
2,Free tour por Oviedo,Déjate seducir por los aires novelescos de la ...,¡Gratis!
3,Free tour por Gijón,Disfruta del patrimonio y de la historia deuna...,¡Gratis!
4,"Excursión a Llanes, Ribadesella y San Vicente ...","En estaexcursióna Llanes, Ribadesella ySan Vic...",35€
5,"Excursión a Covadonga, Cangas de Onís y Lastres",Dejaos cautivar por labelleza asturianaen esta...,42€
6,Visita guiada por la catedral de Oviedo,En estavisita por la catedral de Oviedodescubr...,8€
7,Tour en jeep + Senderismo por la Ruta del Cares,LaRuta del Carespermite adentrarse en los bell...,40€
8,Degustación de sidra y quesos en Cuevas del Agua,En estadegustación de sidra y quesos en Cuevas...,5€
9,Excursión a los lagos de Covadonga,Descubriréis lanaturaleza de los Picos de Euro...,30€


In [21]:
df_actividades.to_csv('../Datos/actividades_asturias.csv')

Vamos a tratar los datos para dejarlos en el formato correcto y que surjan problemas a la hora del analisis

In [23]:
df_actividades = pd.read_csv('../Datos/actividades_asturias.csv')
df_actividades.head()


,Unnamed: 0,Actividad,Descripción,Precio
0,0,"Excursión a Covadonga, Cangas de Onís y la cos...","Si os apasionala historia y la naturaleza, no ...",35€
1,1,"Excursión a Avilés, Cudillero y Luarca",Acompañadnos a descubrir el hermoso ycautivado...,35€
2,2,Free tour por Oviedo,Déjate seducir por los aires novelescos de la ...,¡Gratis!
3,3,Free tour por Gijón,Disfruta del patrimonio y de la historia deuna...,¡Gratis!
4,4,"Excursión a Llanes, Ribadesella y San Vicente ...","En estaexcursióna Llanes, Ribadesella ySan Vic...",35€


In [43]:
df_actividades.dtypes


Actividad      object
Descripción    object
Precio         object
dtype: object

In [58]:
## pasamos a float el precio
df_actividades['Precio'] = df_actividades['Precio'].replace('¡Gratis!', '0')
df_actividades.head(5)

,Actividad,Descripción,Precio
0,"Excursión a Covadonga, Cangas de Onís y la cos...","Si os apasionala historia y la naturaleza, no ...",35.0
1,"Excursión a Avilés, Cudillero y Luarca",Acompañadnos a descubrir el hermoso ycautivado...,35.0
2,Free tour por Oviedo,Déjate seducir por los aires novelescos de la ...,0.0
3,Free tour por Gijón,Disfruta del patrimonio y de la historia deuna...,0.0
4,"Excursión a Llanes, Ribadesella y San Vicente ...","En estaexcursióna Llanes, Ribadesella ySan Vic...",35.0


In [61]:
df_actividades['Precio'] = df_actividades['Precio'].str.replace('€', '')
df_actividades.head(2)


AttributeError: Can only use .str accessor with string values!

In [66]:

df_actividades['Precio'] = df_actividades['Precio'].astype(float)
df_actividades

,Actividad,Descripción,Precio
0,"Excursión a Covadonga, Cangas de Onís y la cos...","Si os apasionala historia y la naturaleza, no ...",35.0
1,"Excursión a Avilés, Cudillero y Luarca",Acompañadnos a descubrir el hermoso ycautivado...,35.0
2,Free tour por Oviedo,Déjate seducir por los aires novelescos de la ...,0.0
3,Free tour por Gijón,Disfruta del patrimonio y de la historia deuna...,0.0
4,"Excursión a Llanes, Ribadesella y San Vicente ...","En estaexcursióna Llanes, Ribadesella ySan Vic...",35.0
5,"Excursión a Covadonga, Cangas de Onís y Lastres",Dejaos cautivar por labelleza asturianaen esta...,42.0
6,Visita guiada por la catedral de Oviedo,En estavisita por la catedral de Oviedodescubr...,8.0
7,Tour en jeep + Senderismo por la Ruta del Cares,LaRuta del Carespermite adentrarse en los bell...,40.0
8,Degustación de sidra y quesos en Cuevas del Agua,En estadegustación de sidra y quesos en Cuevas...,5.0
9,Excursión a los lagos de Covadonga,Descubriréis lanaturaleza de los Picos de Euro...,30.0


In [67]:
df_actividades.to_csv('../Datos/actividades_asturias.csv')